In [1]:
import cv2
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import os
import numpy as np
from preprocessing import *
from skimage.morphology import skeletonize

In [2]:
def dist(x, y, x1, y1):
    return ((x - x1)**2)**(0.5) + ((y - y1)**2) ** (0.5)

def getThicknessHist(skeleton,binarizedImg):
    blackPixelsSk = np.where(skeleton==0)
    contours,_ = cv2.findContours(255-binarizedImg, cv2.RETR_EXTERNAL,  cv2.CHAIN_APPROX_NONE)
    boundindBoxes = [cv2.boundingRect(contour) for contour in contours]
    thick_arr = []
    for i in range(len(blackPixelsSk[0])):
        y,x = blackPixelsSk[0][i],blackPixelsSk[1][i]

        for idx,(xB,yB,wB,hB) in enumerate(boundindBoxes):
            if y>=yB and x>=xB and y<=(yB+hB) and x<=(xB+wB):
                insideContour = contours[idx] 
                break

        insideContour = insideContour.T

        cols= insideContour[0][0]
        rows= insideContour[1][0]
        idxs = list(range(len(rows)))
        abovePoints = []
        belowPoints = []
        rightPoints = []
        leftPoints =  []
        
        for i,r,c in zip(idxs,rows,cols):
            if r<y and c==x:
                abovePoints.append(i)
            elif  r>y and c==x:
                belowPoints.append(i)
            elif r==y and c>x:
                rightPoints.append(i)
            elif r==y and c<x:
                leftPoints.append(i)
                
        if len(belowPoints):
            minIdx = np.argmin([dist(x,y,cols[belowPoints[i]],rows[belowPoints[i]]) for i in range(len(belowPoints))])
            nearBelow = rows[belowPoints[minIdx]],cols[belowPoints[minIdx]] 
        else: nearBelow = y,x

        if len(abovePoints):
            minIdx = np.argmin([dist(x,y,cols[abovePoints[i]],rows[abovePoints[i]]) for i in range(len(abovePoints))])
            nearAbove = rows[abovePoints[minIdx]],cols[abovePoints[minIdx]] 
        else: nearAbove = y,x

        if len(rightPoints): 
            minIdx = np.argmin([dist(x,y,cols[rightPoints[i]],rows[rightPoints[i]]) for i in range(len(rightPoints))])
            nearRight = rows[rightPoints[minIdx]],cols[rightPoints[minIdx]] 
        else: nearRight = y,x

        if len(leftPoints): 
            minIdx = np.argmin([dist(x,y,cols[leftPoints[i]],rows[leftPoints[i]]) for i in range(len(leftPoints))])
            nearLeft = rows[leftPoints[minIdx]],cols[leftPoints[minIdx]] 
        else: nearLeft = y,x
            
        distVer = dist(nearBelow[1],nearBelow[0],nearAbove[1],nearAbove[0])
        distHor = dist(nearRight[1],nearRight[0],nearLeft[1],nearLeft[0])
        thickness = min(distVer,distHor)
        thick_arr.append(thickness)
    thick_hist,bins = np.histogram(thick_arr, 10)
    return list(thick_hist),list(bins)
    

In [3]:
def Train_NN(X,Y):
    print("Start Training ")
    N = len(X[0])
    X = np.array(X)
    Y = np.array(Y)
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=4, stratify=Y)
    clf = MLPClassifier(alpha=1e-05, hidden_layer_sizes=(32,16),random_state=1,solver='lbfgs',max_iter=10000)
    clf = clf.fit(x_train, y_train)
    print("Finished Fitting")
    y_pred_test = clf.predict(x_test)
    accuracy_test = np.mean(y_test==y_pred_test) * 100
    
    y_pred_train = clf.predict(x_train)
    accuracy_train = np.mean(y_train==y_pred_train) * 100
    return accuracy_train,accuracy_test,clf

In [ ]:

base_dir='ACdata_base/'

fonts = os.listdir(base_dir)
X,Y=[],[]
for font in fonts:
    h,w = 0,0
    data = os.listdir(base_dir+font)
    print("curFont",font)
    for img in data:
        img_dir = base_dir+font+'/'+img
        imgGray = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
        binarizedImg =  Binarize_Histogram(imgGray,'')
        skeleton = 255 - skeletonize(1-binarizedImg/255)*255
        hist,bins = getThicknessHist(skeleton,binarizedImg)
        features = hist + bins
#         print(features)
        X.append(features)
        Y.append(int(font))


curFont 8


In [ ]:
accuracyTrain,accuracyTest,ModelNN = Train_NN(X,Y)

In [ ]:
imgGray = cv2.imread('1498.jpg', cv2.IMREAD_GRAYSCALE)
binarizedImg =  Binarize_Histogram(imgGray,'')
skeleton = 255 - skeletonize(1-binarizedImg/255)*255
hist,bins = getThicknessHist(skeleton,binarizedImg)
plt.hist(bins[:-1], bins, weights=hist)
print(hist)
print(bins)